In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atrial_Fibrillation"
cohort = "GSE143924"

# Input paths
in_trait_dir = "../../input/GEO/Atrial_Fibrillation"
in_cohort_dir = "../../input/GEO/Atrial_Fibrillation/GSE143924"

# Output paths
out_data_file = "../../output/preprocess/Atrial_Fibrillation/GSE143924.csv"
out_gene_data_file = "../../output/preprocess/Atrial_Fibrillation/gene_data/GSE143924.csv"
out_clinical_data_file = "../../output/preprocess/Atrial_Fibrillation/clinical_data/GSE143924.csv"
json_path = "../../output/preprocess/Atrial_Fibrillation/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis from human epicardial adipose tissue biopsies analyzed according to postoperative atrial fibrillation occurrence after cardiac surgery"
!Series_summary	"Introduction: Post-operative atrial fibrillation (POAF) is a frequent complication after cardiac surgery, but its pathophysiology remains incompletely understood. Considering that epicardial adipose tissue (EAT) is in close vicinity with the atrial myocardium, we hypothesized that a specific pre-operative EAT phenotype would be associated to POAF onset following surgery.  Methods: Patients undergoing cardiac surgery prospectively enrolled in the POMI-AF cohort between February 2016 and June 2017 were studied. EAT samples were collected at the beginning of surgery. Whole-tissue gene expression patterns and the stromal and vascular fraction (SVF) cellular composition were explored. Patients were followed after surgery by continuous ECG to detect POAF onset.  Results: Among the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Define variables and conversion functions for dataset analysis

# 1. Gene Expression Data Availability
# Based on the description, this dataset contains transcriptome analysis data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (AF): identified in row 1 of characteristics dictionary
trait_row = 1
# Age and gender information is not provided in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(x):
    """Convert trait data to binary format (0: no AF, 1: AF)"""
    if ':' in x:
        value = x.split(':', 1)[1].strip()
        if 'sinus rhythm' in value.lower():
            return 0  # No atrial fibrillation
        elif 'atrial fibrillation' in value.lower() or 'poaf' in value.lower():
            return 1  # Has atrial fibrillation
    return None

def convert_age(x):
    """Convert age data to continuous format"""
    # Not applicable for this dataset
    return None

def convert_gender(x):
    """Convert gender data to binary format (0: female, 1: male)"""
    # Not applicable for this dataset
    return None

# 3. Save Metadata
# Initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # Convert the sample characteristics dictionary to a DataFrame format
    sample_chars = {0: ['tissue: epicardial adipose tissue'], 
                    1: ['patient diagnosis: sinus rhythm after surgery', 
                        'patient diagnosis: postoperative atrial fibrillation after surgery (POAF)']}
    
    # Create a DataFrame from the sample characteristics
    # First, find the maximum number of values in any row
    max_values = max(len(values) for values in sample_chars.values())
    
    # Create a dictionary to hold the transformed data
    clinical_dict = {}
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            col_name = f"characteristic_{col_idx+1}"
            if col_name not in clinical_dict:
                clinical_dict[col_name] = [None] * len(sample_chars)
            clinical_dict[col_name][row_idx] = value
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame(clinical_dict)
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the extracted clinical features as a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'characteristic_1': [0.0], 'characteristic_2': [1.0]}
Clinical data saved to ../../output/preprocess/Atrial_Fibrillation/clinical_data/GSE143924.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atrial_Fibrillation/GSE143924/GSE143924_series_matrix.txt.gz
Gene data shape: (8596, 30)
First 20 gene/probe identifiers:
Index(['A2M-AS1', 'AACS', 'AADAC', 'AADACL2', 'AADACP1', 'AAK1', 'AARD',
       'AASS', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA3', 'ABCA5', 'ABCA9-AS1',
       'ABCB1', 'ABCB11', 'ABCB4', 'ABCB7', 'ABCC3', 'ABCC6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers from the output provided
# The identifiers shown (A2M-AS1, AACS, AADAC, etc.) appear to be 
# standard human gene symbols rather than probe IDs or other identifiers

# These match the standard HGNC (HUGO Gene Nomenclature Committee) gene symbol format
# Examples like ABCA1, ABCC3 are well-known human gene symbols
# Identifiers like A2M-AS1 and ABCA9-AS1 follow the antisense RNA naming convention

# No mapping needed as these are already proper human gene symbols
requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (8596, 30)
Gene data shape after normalization: (7873, 30)


Normalized gene expression data saved to ../../output/preprocess/Atrial_Fibrillation/gene_data/GSE143924.csv


Original clinical data preview:
         !Sample_geo_accession                                     GSM4276706  \
0  !Sample_characteristics_ch1              tissue: epicardial adipose tissue   
1  !Sample_characteristics_ch1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276707  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276708  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276709  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276710  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276711  \
0              tissue: epicardial adipose ti

Data shape after handling missing values: (30, 7874)
For the feature 'Atrial_Fibrillation', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.


Linked data saved to ../../output/preprocess/Atrial_Fibrillation/GSE143924.csv
